# WORK IN PROGRESS 
this section is dedicated to mamba models, this is going to be a naive and slow implementation in pytorch, optimized for understanding of the underlying concepts. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt 
device = torch.device("cpu")


In [2]:
with open("datasets/tinyShakespeare.txt", "r") as f:
    load = f.read()
print(load[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
texttoint = {elm: n for n, elm in enumerate(set(load))}
inttotext = {n:elm for n, elm in enumerate(set(load))}
dataset = [texttoint[c] for c in load]

vocab_size = len(texttoint)
sequence_length = 20

sequences = torch.tensor([dataset[i:-sequence_length+i-1] for i in range(sequence_length+1)]).T
inputs = sequences[2000:, :-1]
labels = sequences[2000:, -1]
val_inputs = sequences[:2000, :-1]
val_labels = sequences[:2000, -1]

In [4]:
class LinearRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LinearRNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size).to(device)

In [5]:
lin = torch.rand(10, 128)

naive_sum = torch.stack([sum(lin[:i]) for i in range(1, len(lin)+1)])

class tinyMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(tinyMLP, self).__init__()
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = F.relu(self.i2h(x))
        x = self.h2o(x)
        return x

myMlp = tinyMLP(128, 256, 128)

elementwise_mlp = torch.stack([myMlp(lin[i]) for i in range(len(lin))])
elementwise_mlp

tensor([[ 0.0383,  0.0113,  0.1586,  ...,  0.0284,  0.1084, -0.0956],
        [ 0.0294,  0.0217,  0.0982,  ..., -0.0601,  0.0698,  0.0708],
        [ 0.0619, -0.0381,  0.2825,  ..., -0.1390,  0.1544, -0.0848],
        ...,
        [ 0.1536,  0.0590,  0.1152,  ...,  0.0081,  0.0987, -0.0990],
        [ 0.1256, -0.0770,  0.1430,  ...,  0.0122,  0.0611,  0.0072],
        [ 0.0947, -0.0232,  0.1180,  ..., -0.0124,  0.1136, -0.1291]],
       grad_fn=<StackBackward0>)

In [6]:
class recursiveBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(recursiveBlock, self).__init__()
        n = 3
        self.linear_in = nn.Linear(input_size, hidden_size)
        self.linear_hidden = [nn.Linear(hidden_size, hidden_size) for _ in range(n)]
        self.linear_out = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        naive_sum = torch.stack([sum(x[:i]) for i in range(1, len(x)+1)])
        # naive_sum = 

        x = F.relu(self.linear_in(naive_sum))
        for layer in self.linear_hidden:
            x = F.relu(layer(x))
        x = self.linear_out(x)
        return x
    

class recursiveNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(recursiveNetwork, self).__init__()
        self.block = recursiveBlock(input_size, hidden_size)
    
    def forward(self, x):
        for _ in range(3):
            x += self.block(x)
        return x

In [7]:
lin = torch.rand(10, 128)

myBlock = recursiveBlock(128, 128)

out = myBlock(lin)

In [8]:
# Define hyperparameters
input_size = vocab_size
hidden_size = 128
output_size = vocab_size
learning_rate = 0.001
num_epochs = 1000
batch_size = 128

# Initialize the model, loss function, and optimizer
emb = nn.Embedding(vocab_size, input_size)
pemb = nn.Embedding(sequence_length, input_size)
model = recursiveNetwork(input_size, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lossi = []

# Training loop
for epoch in range(num_epochs):
    idx = torch.randperm(inputs.shape[0])[:batch_size]
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    batch_inputs = inputs[idx].to(device)
    batch_labels = labels[idx].to(device)

    embeddings = emb(batch_inputs) + pemb(torch.arange(sequence_length-1, -1, -1).to(device))
    outputs = model(embeddings)
    loss = criterion(outputs.view(batch_size, 20, vocab_size)[:, -1], batch_labels.long())
    lossi.append(loss.item())

    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Validation
model.eval()
with torch.no_grad():
    embeddings = emb(val_inputs).to(device) + pemb(torch.arange(sequence_length-1, -1, -1).to(device))
    val_outputs = model(embeddings)
    val_loss = criterion(val_outputs.view(val_inputs.shape[0], 20, vocab_size)[:, -1], val_labels.long())
    print(f'Validation Loss: {val_loss.item():.4f}')

Epoch [10/1000], Loss: 5.0018
Epoch [20/1000], Loss: 4.1829
Epoch [30/1000], Loss: 4.1310
Epoch [40/1000], Loss: 4.1113
Epoch [50/1000], Loss: 4.0913
Epoch [60/1000], Loss: 4.1256


KeyboardInterrupt: 